 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="https://rnap-nm.ddns.net/jupyterhub/user/yugo4k/notebooks/yugo4k/rnap-cs3/PyTorch/livros_dataset.ipynb#Classificação-de-Textos" data-toc-modified-id="Classificação-de-Textos-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Classificação de Textos</a></span><ul class="toc-item"><li><span><a href="https://rnap-nm.ddns.net/jupyterhub/user/yugo4k/notebooks/yugo4k/rnap-cs3/PyTorch/livros_dataset.ipynb#Preâmbulo" data-toc-modified-id="Preâmbulo-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Preâmbulo</a></span></li><li><span><a href="https://rnap-nm.ddns.net/jupyterhub/user/yugo4k/notebooks/yugo4k/rnap-cs3/PyTorch/livros_dataset.ipynb#Preparando-o-dataset" data-toc-modified-id="Preparando-o-dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Preparando o dataset</a></span><ul class="toc-item"><li><span><a href="https://rnap-nm.ddns.net/jupyterhub/user/yugo4k/notebooks/yugo4k/rnap-cs3/PyTorch/livros_dataset.ipynb#Buscando-o-texto-dos-livros-e-definindo-os-rótulos" data-toc-modified-id="Buscando-o-texto-dos-livros-e-definindo-os-rótulos-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Buscando o texto dos livros e definindo os rótulos</a></span></li><li><span><a href="https://rnap-nm.ddns.net/jupyterhub/user/yugo4k/notebooks/yugo4k/rnap-cs3/PyTorch/livros_dataset.ipynb#Representando-as-palavras-através-de-índices-inteiros" data-toc-modified-id="Representando-as-palavras-através-de-índices-inteiros-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Representando as palavras através de índices inteiros</a></span></li><li><span><a href="https://rnap-nm.ddns.net/jupyterhub/user/yugo4k/notebooks/yugo4k/rnap-cs3/PyTorch/livros_dataset.ipynb#Palavras-características-de-cada-livro" data-toc-modified-id="Palavras-características-de-cada-livro-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Palavras características de cada livro</a></span></li></ul></li><li><span><a href="https://rnap-nm.ddns.net/jupyterhub/user/yugo4k/notebooks/yugo4k/rnap-cs3/PyTorch/livros_dataset.ipynb#Dividindo-o-dataset-entre-treinamento-e-validação" data-toc-modified-id="Dividindo-o-dataset-entre-treinamento-e-validação-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Dividindo o dataset entre treinamento e validação</a></span><ul class="toc-item"><li><span><a href="https://rnap-nm.ddns.net/jupyterhub/user/yugo4k/notebooks/yugo4k/rnap-cs3/PyTorch/livros_dataset.ipynb#Divisão-simples" data-toc-modified-id="Divisão-simples-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Divisão simples</a></span></li><li><span><a href="https://rnap-nm.ddns.net/jupyterhub/user/yugo4k/notebooks/yugo4k/rnap-cs3/PyTorch/livros_dataset.ipynb#Divisão-para-uso-com-geradores-e-aumento-de-dados" data-toc-modified-id="Divisão-para-uso-com-geradores-e-aumento-de-dados-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Divisão para uso com geradores e aumento de dados</a></span><ul class="toc-item"><li><span><a href="https://rnap-nm.ddns.net/jupyterhub/user/yugo4k/notebooks/yugo4k/rnap-cs3/PyTorch/livros_dataset.ipynb#Criando--geradores-para-treino-e-validação" data-toc-modified-id="Criando--geradores-para-treino-e-validação-1.3.2.1"><span class="toc-item-num">1.3.2.1&nbsp;&nbsp;</span>Criando  geradores para treino e validação</a></span></li></ul></li></ul></li></ul></li></ul></div>

# Classificação de Textos

## Preâmbulo

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display

import os
import sys
import imp
import time
import glob
import numpy as np
import numpy.random as nr

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR, StepLR
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable

import torchvision as tv
import lib.pytorch_trainer as ptt
from lib.tokenizer import Tokenizer

use_gpu = torch.cuda.is_available()
print('GPU available:', use_gpu)
print('torch', torch.version.__version__)
print('Python', sys.version)

np.set_printoptions(precision=3, linewidth=120)

GPU available: False
torch 0.2.0_3
Python 3.6.2 (default, Sep 28 2017, 13:18:00) 
[GCC 5.4.0 20160609]


## Preparando o dataset

### Buscando o texto dos livros e definindo os rótulos

In [2]:
data_dir = '/data/datasets/livros/'

autores = [
    'Jorge_Amado',
    'Machado_de_Assis',
    'Érico_Veríssimo',
]

book_text = []
book_author = []
book_title = []
for aut in autores:
    for fn in glob.glob(data_dir + 'processed/' + aut + '*.txt'):
        author, book = os.path.basename(fn).split('__')
        txt = open(fn, encoding='utf-8').read().replace('\x97', '')
        book_text.append(txt)
        book_author.append(author)
        book_title.append(book[:-4])
        print('{:7d}  {:18s} {}'.format(len(txt), author, book[:-4]))

author_list = list(set(book_author))
n_labels = len(author_list)
n_books = len(book_title)
book_label = [author_list.index(a) for a in book_author]
print('\n{} Labels:'.format(n_labels))
for i, autor in enumerate(author_list):
    print('    {:2d}: {}'.format(i, autor))

1030735  Jorge_Amado        Dona_flor_seus_dois_maridos
 828417  Jorge_Amado        Gabriela
 427711  Jorge_Amado        Capitães_de_Areia
1001226  Jorge_Amado        Tereza_Batista_Cansada_de_Guerra
 372459  Machado_de_Assis   Dom_Casmurro
 443778  Machado_de_Assis   Quincas_Borba
 337533  Machado_de_Assis   Helena
 336677  Machado_de_Assis   Iaiá_Garcia
 411043  Machado_de_Assis   Esaú_e_Jacó
 352965  Machado_de_Assis   Memórias_Póstumas_de_Brás_Cubas
 280683  Machado_de_Assis   Memorial_de_Aires
 749265  Érico_Veríssimo    O_Tempo_e_o_Vento_-_O_Continente
 890215  Érico_Veríssimo    Incidente_em_Antares
 294049  Érico_Veríssimo    Clarissa
 699390  Érico_Veríssimo    O_Tempo_e_o_Vento_-_O_Arquipélago

3 Labels:
     0: Machado_de_Assis
     1: Érico_Veríssimo
     2: Jorge_Amado


### Representando as palavras através de índices inteiros

In [3]:
MAX_NB_WORDS = 20000

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(book_text)
sequences = tokenizer.texts_to_sequences(book_text)

w2i = tokenizer.word_index
i2w = dict([(v, k) for k, v in w2i.items()])

i2w_vec = np.array([i2w[i+1] for i in range(len(i2w))])

print('Found %d unique tokens.' % len(i2w))
print('Using the first %d words.' % max([max(s) for s in sequences]))

Found 61707 unique tokens.
Using the first 19999 words.


In [4]:
# escolhe e imprime uma sequencia aleatoria de cada livro
rseq_len = 100
for i, seq in enumerate(sequences):
    k = nr.randint(len(seq) - rseq_len)
    print('{}: {} -- {} palavras'.format(book_author[i], book_title[i], len(seq)).replace('_', ' '))
    print(' '.join([i2w[x] for x in seq[k:k+rseq_len]]), end='\n\n')


Jorge Amado: Dona flor seus dois maridos -- 166625 palavras
e as más recordações todas elas a a carregar o nojo a transpor aquele tempo gris de desespero e ausência um deserto de cinzas mesmo ao rever lembranças e imagens tão como a da ex aluna com seu riso e sua cínica mesmo ao ferir se novamente em tais espinhos ao aquelas humilhações sentia uma espécie de agre consolo como se lembranças e imagens espinhos e humilhações tudo quanto vivera com ele fosse para esse sofrimento esse de agora sem medida e sem jeito porque afinal quem saíra vitoriosa quem vence ra a parada quem ficara com ele por quem

Jorge Amado: Gabriela -- 134498 palavras
eu lhe digo minha filha é só você querer saber levar as coisas com jeito dando e negando deixando ele com água na boca ele já anda assustado pág m eu chico me contou que o juiz fala em botar casa pra você ele ouviu seu nhô galo dizer seu nacib anda com o coração na mão quero não morria o sorriso em seus lábios gosto dele não velho sem graça esse tal de

### Palavras características de cada livro

In [5]:
tfidf = tokenizer.sequences_to_matrix(sequences, mode='tfidf')
ww = np.argsort(tfidf, axis=1)[:, -8:]
print(i2w_vec[ww-1])

[['marilda' 'dinorá' 'teodoro' 'pelancchi' 'gisa' 'mirandão' 'rozilda' 'vadinho']
 ['tuísca' 'tonico' 'ribeirinho' 'malvina' 'amâncio' 'fulgêncio' 'gabriela' 'nacib']
 ['trapiche' 'bedel' 'ester' 'barandão' 'almiro' 'dora' '–' 'pirulito']
 ['vavá' 'januário' 'almério' '—' 'dóris' 'brígida' 'justiniano' 'tereza']
 ['manduca' 'protonotário' 'bentinho' 'sancha' 'pádua' 'justina' 'escobar' 'capitu']
 ['camacho' 'teófilo' 'tonica' 'borba' 'sofia' 'fernanda' 'benedita' 'rubião']
 ['ângela' 'eugênia' 'tomásia' 'melchior' 'helena' 'camargo' 'estácio' 'úrsula']
 ['procópio' 'valéria' 'jorge' 'garcia' 'madrasta' 'enteada' 'iaiá' 'estela']
 ['coupé' 'gêmeos' 'excia' 'custódio' 'nóbrega' 'natividade' 'flora' 'cláudia']
 ['loló' 'eusébia' 'cubas' 'borba' 'sabina' 'cotrim' 'marcela' 'virgília']
 ['carmo' 'libertos' 'prainha' 'noronha' 'cesária' 'aguiar' 'fidélia' 'tristão']
 ['maneco' 'amaral' 'lara' 'vosmecê' 'bibiana' '—' 'alonzo' 'rodrigo']
 ['campolargo' 'vacariano' 'vivaldino' 'getúlio' 'quitér

## Dividindo o dataset entre treinamento e validação

In [6]:
nr.seed(20170607)

batch_size  = 32
seq_size    = 50
valid_split = 0.2

### Divisão simples

In [7]:
all_data = [[] for i in range(n_labels)]

# divide cada livro em sequencias de 'seq_size' words
# 'all_data' contem as sequencias agrupadas por autor
for sequence, label in zip(sequences, book_label):
    n_seqs = len(sequence) // seq_size
    for i in range(n_seqs):
        beg = i * seq_size
        all_data[label].append(sequence[beg:beg+seq_size])

print('Sequencias obtidas para cada autor:', [len(x) for x in all_data])

# balanceando o dataset:
# calcula o numero de sequencias, N, de forma que o dataset 
# contenha N sequencias para cada autor
N = min([len(x) for x in all_data])
print('O dataset conterá {} sequencias por autor, totalizando {} sequencias.'.format(N, 3*N))

all_data = np.array([seq[:N] for seq in all_data], np.int32).reshape(-1, seq_size)
all_labels = np.array([[i] * N for i in range(n_labels)], np.int32).reshape(-1)
print('\nDataset shapes:', all_data.shape, all_labels.shape)


Sequencias obtidas para cada autor: [8564, 8747, 10841]
O dataset conterá 8564 sequencias por autor, totalizando 25692 sequencias.

Dataset shapes: (25692, 50) (25692,)


In [8]:
from sklearn.model_selection import train_test_split

Xtra, Xval, ytra, yval = train_test_split(all_data, all_labels, test_size=valid_split)
print(Xtra.shape, ytra.shape, Xval.shape, yval.shape)

(20553, 50) (20553,) (5139, 50) (5139,)


In [9]:
fn = data_dir + 'livros_sequences_{}.npz'.format(seq_size)
np.savez_compressed(fn, Xtra=Xtra, Xval=Xval, ytra=ytra, yval=yval, i2w=i2w_vec[:MAX_NB_WORDS])

### Divisão para uso com geradores e aumento de dados

In [10]:
valid_length = [int(0.2 * len(x)) for x in sequences]
valid_start = [nr.randint(2000, len(x) - 2000 - n) for x, n in zip(sequences, valid_length)]

valid_sequences = [seq[x0:x0+n] for seq, x0, n in zip(sequences, valid_start, valid_length)]

train_sequences = [seq[:x0] for seq, x0 in zip(sequences, valid_start)] + \
                  [seq[x0+n:] for seq, x0, n in zip(sequences, valid_start, valid_length)]

valid_labels = book_label
train_labels = book_label + book_label

n_train_words = sum([len(x) for x in train_sequences])
n_valid_words = sum([len(x) for x in valid_sequences])

print('Training sequences:')
print('-------------------')
for i, (seq, lab) in enumerate(zip(train_sequences, train_labels)):
    if i < n_books:
        print('{:2d}. {:16s} ({}) -- {:6d} palavras do início do livro {}'.format(i+1, book_author[i%n_books], lab,
                                                                                  len(seq), book_title[i%n_books]))
    else:
        print('{:2d}. {:16s} ({}) -- {:6d} palavras do final do livro  {}'.format(i+1, book_author[i%n_books], lab,
                                                                                  len(seq), book_title[i%n_books]))
print()
print('Validation sequences:')
print('---------------------')
for i, (seq, lab) in enumerate(zip(valid_sequences, valid_labels)):
    print('{:2d}. {:16s} ({}) -- {:6d} palavras do meio do livro {}'.format(i+1, book_author[i%n_books], lab,
                                                                            len(seq), book_title[i%n_books]))
print()
print('Total number of training words:  ', n_train_words)
print('Total number of validation words:', n_valid_words)

Training sequences:
-------------------
 1. Jorge_Amado      (2) -- 116879 palavras do início do livro Dona_flor_seus_dois_maridos
 2. Jorge_Amado      (2) --  58438 palavras do início do livro Gabriela
 3. Jorge_Amado      (2) --  34178 palavras do início do livro Capitães_de_Areia
 4. Jorge_Amado      (2) --  33045 palavras do início do livro Tereza_Batista_Cansada_de_Guerra
 5. Machado_de_Assis (0) --   3266 palavras do início do livro Dom_Casmurro
 6. Machado_de_Assis (0) --  37666 palavras do início do livro Quincas_Borba
 7. Machado_de_Assis (0) --  10817 palavras do início do livro Helena
 8. Machado_de_Assis (0) --  33350 palavras do início do livro Iaiá_Garcia
 9. Machado_de_Assis (0) --   3461 palavras do início do livro Esaú_e_Jacó
10. Machado_de_Assis (0) --  11576 palavras do início do livro Memórias_Póstumas_de_Brás_Cubas
11. Machado_de_Assis (0) --  26516 palavras do início do livro Memorial_de_Aires
12. Érico_Veríssimo  (1) --  22847 palavras do início do livro O_

#### Criando  geradores para treino e validação

In [11]:
class MyDataGenerator:
    def __init__(self, batch_size, seq_size, sequences, labels):
        self.batch_size = batch_size
        self.length = seq_size
        self.sequences = sequences
        self.labels = labels
        sizes = np.array([len(seq) for seq in sequences])
        self.p = 1.0 * sizes / sizes.sum()        # probabilidade de escolha para cada sequencia
        self.n = np.arange(len(sequences))        # indices de cada sequencia (para o choice abaixo)
        
    def __call__(self):
        while True:
            batch = np.empty((self.batch_size, self.length), np.int32)
            label = np.empty((self.batch_size, n_labels), np.int32)
            for i in range(self.batch_size):
                k = nr.choice(self.n, p=self.p)
                p = nr.randint(0, len(self.sequences[k]) - self.length)
                batch[i] = self.sequences[k][p:p+self.length]
                label[i] = to_categorical(self.labels[k], num_classes=n_labels)
            yield batch, label

            
train_gen = MyDataGenerator(batch_size, seq_size, train_sequences, train_labels)
valid_gen = MyDataGenerator(batch_size, seq_size, valid_sequences, valid_labels)


In [12]:
import pickle
fn = data_dir + 'livros_generators_{}.pkl'.format(seq_size)
pickle.dump([train_gen, valid_gen, i2w_vec[:MAX_NB_WORDS]], open(fn, 'wb'))